In [3]:
import pandas as pd
from proj_acquire import *
from proj_prepare import *

In [4]:
data = acquire_fitbit()

In [5]:
data = prepare_fitbit()

In [6]:
data

,index,activity_calories,distance,floors,mins_active_high,mins_active_light,mins_active_med,mins_sedentary,steps,total_calories,calories,carbs,fat,fiber,protien,sodium,water
0,2018-04-26,566,1.38,2,0,93,0,"1,347","2,931","2,635",0,0 g,0 g,0 g,0 g,0 mg,0 fl oz
1,2018-04-27,"1,752",4.7,19,17,219,46,"1,158","9,971","3,649",0,0 g,0 g,0 g,0 g,0 mg,0 fl oz
2,2018-04-28,"1,124",1.66,2,11,155,22,702,"3,521","3,379",0,0 g,0 g,0 g,0 g,0 mg,0 fl oz
3,2018-04-29,602,0.77,0,0,122,0,891,"1,639","2,851",0,0 g,0 g,0 g,0 g,0 mg,0 fl oz
4,2018-04-30,540,0.62,0,0,104,0,880,"1,323","2,825",0,0 g,0 g,0 g,0 g,0 mg,0 fl oz
5,2018-05-01,"1,068",1.22,0,2,167,13,919,"2,581","3,292",0,0 g,0 g,0 g,0 g,0 mg,0 fl oz
6,2018-05-02,419,0.65,0,0,90,0,927,"1,372","2,660",0,0 g,0 g,0 g,0 g,0 mg,0 fl oz
7,2018-05-03,"1,006",1.05,0,0,157,16,876,"2,232","3,121",0,0 g,0 g,0 g,0 g,0 mg,0 fl oz
8,2018-05-04,269,0.43,1,0,61,0,"1,012",904,"2,522",0,0 g,0 g,0 g,0 g,0 mg,0 fl oz
9,2018-05-05,326,0.39,0,0,68,0,969,833,"2,551",0,0 g,0 g,0 g,0 g,0 mg,0 fl oz
